Convolution
- a filter of weights that are used to mulyiply a pixel with its neighbours to get a new value for the pixel
- With this, we can potentially learn a set of filters that reduce the image to features. Combined with pooling, we can reduce the amount of information in the image while maintaining the features

Eg:
Image pixels             Filter
[[0, 64, 128],          [[-1, 0, -2],
 [48, 192, 144],         [0.5, 4.5, -1.5],
 [142, 226, 168]]        [1.5, 2, -3]]

Convolution formula = (-1 * 0) + (0 * 64) + (-2 * 128) +
                      (0.5 * 48) + (4.5 * 192) + (-1.5 * 144) +
                      (1.5 * 142) + (2 * 226) + (-3 * 168)
- This equals to 577 which will be the new value for that pixel. Repeating this process across every pixel in the image will give us a filtered image

Pooling
- Pooling is the process of eliminating picels in the images while maintaining the semantics of the content within the image.

1) Max pooling
Image pixel (Total of 16 pixels)
[[0, 64, 128, 128],
 [48, 192, 144, 144],
 [142, 226, 168, 0],
 [255, 0, 0, 64]
]

Then the pooling algorithm seperate the pixels into four equal parts, then the maximum value for each groups are selected
[[0, 64],         [[128, 128],          [[142, 226]         [[168, 0],          
 [48, 192]]        [144, 144]]           [255, 0]]           [0, 64]]

New pixel = [
    [192, 144],
    [255, 168]
]

- The filtered features is further enhanced and also, the image size has changed from 512 * 512 to 256 * 256

In [2]:
# Implementing CNN with max pooling
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.nn import relu, softmax
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.callbacks import Callback

In [3]:
# Callback class
class myCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.95):
            print("\nAccuracy reached 95%!!! Stopping Training....")
            self.model.stop_training = True 

In [4]:
data = fashion_mnist
callbacks = myCallback()

(training_features, training_labels), (testing_features, testing_labels) = data.load_data()

# Reshaping the data so it fits into the model
training_features = training_features.reshape(60000, 28, 28, 1)
training_features = training_features / 255.0
testing_features = testing_features.reshape(10000, 28, 28, 1)
testing_features = testing_features / 255.0


# Creating Model
model = Sequential([
    Conv2D(64, (3, 3), activation = 'relu', input_shape = (28, 28, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation = 'relu', input_shape = (28, 28, 1)),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation = relu),
    Dense(10, activation = softmax)

])

# Compiling Model
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# Training Model
model.fit(training_features, training_labels, epochs = 50, callbacks = [callbacks])

model.evaluate(testing_features, testing_labels)

classifications = model.predict(testing_features)
print(f"Model predictions : {classifications[0]}")
print(f"Actual Answer : {classifications[0]}")


Epoch 1/50
1875/1875 [==============================] - 48s 25ms/step - loss: 0.4471 - accuracy: 0.8387
Epoch 2/50
1875/1875 [==============================] - 44s 23ms/step - loss: 0.2971 - accuracy: 0.8897
Epoch 3/50
1875/1875 [==============================] - 43s 23ms/step - loss: 0.2511 - accuracy: 0.9074
Epoch 4/50
1875/1875 [==============================] - 42s 23ms/step - loss: 0.2183 - accuracy: 0.9182
Epoch 5/50
1875/1875 [==============================] - 38s 20ms/step - loss: 0.1913 - accuracy: 0.9275
Epoch 6/50
1875/1875 [==============================] - 39s 21ms/step - loss: 0.1699 - accuracy: 0.9368
Epoch 7/50
1875/1875 [==============================] - 39s 21ms/step - loss: 0.1482 - accuracy: 0.9444
Epoch 8/50
1875/1875 [==============================] - ETA: 0s - loss: 0.1311 - accuracy: 0.9502
Accuracy reached 95%!!! Stopping Training....
313/313 [==============================] - 2s 5ms/step - loss: 0.2647 - accuracy: 0.9102
Model predictions : [1.0078699e-11 2.87

model.summary() 
- returns an overall information about the model

- Our first layer (input layer) will have 28 * 28. But because our filter is 3 * 3, a 1-pixel border around the image will be lost, reducing overall information to 26 * 26
Example:-
口  口  口   o   o   o
口  x   口   o   o   o             口 = filter pixel
口  口  口   o   o   o              x = first pixel to apply filter    
o   o   o   o   o   o              o = remaining pixel
o   o   o   o   o   o
o   o   o   o   o   o
- Thus, an image that is A * B pixels in shape when run through a N1 * N2 filter will become:
(A - [N1 - 1]) * (B - [N2 - 2])

Param #
- parameter is similar to weight in neural network
- becuase the filter is 3 * 3, there are 9 parameters to learn. given that we have 64 convolutions defined, we'll have 640 overall parameters.
(Each convolution has 9 parameters plus a bias, for a total of 10) (64 * [9 + 1])

- The MaxPooling layers don't learn anyhting, they just reduce the image, so there are no learned parameters there - hence 0 is being reported

 

In [7]:
# Inspecting model
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 128)               2